# Rakamin Trial Class Batch 33 - Final Test (Mandatory)
> Author: Nabilah Sharfina

# A. Connect Google Drive

In [ ]:
# Load data from google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# B. Import Library

In [ ]:
import math
from collections import Counter
from pprint import pprint

import pandas as pd # for handling and cleaning the dataset
import numpy as np # for performing mathematical calculations behind ML algorithms
import matplotlib.pyplot as plt # for visualization
import seaborn as sns # for visualization

# One Hot Encoding: digunakan untuk bukan tipe data ordinal, misal nominal, interval, dan ratio
from sklearn.preprocessing import OneHotEncoder
# Ordinal Encoding: digunakan untuk tipe data ordinal
from sklearn.preprocessing import LabelEncoder

# Scalling Data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Handle imbalance dataset
from imblearn.over_sampling import SMOTE

# Splitting Data
from sklearn.model_selection import train_test_split

# Modelling
# !pip install lightgbm
# import lightgbm as lgbm
# from xgboost import XGBClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier

# For feature importance
from sklearn.datasets import make_classification

import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

# For Hyperparameter Tuning
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

# C. Load Data

In [ ]:
# Baca data dari file CSV
df = pd.read_csv("/content/drive/MyDrive/RAKAMIN ACADEMY/TRIAL CLASS/BATCH 33/Dataset Exam/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
# Melihat apakah ada missing value dari dataset
df.isna().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [ ]:
# Mengecek baris yang duplikat
df.duplicated().sum()

1

# D. Section 7-11: Menganalisis Rata-Rata Pendapatan Per Tahun

## 7. Menghitung Nasabah Perempuan yang Merokok (Melakukan Klaim)
> Suatu perusahaan asuransi ingin menghitung berapa banyak nasabah yang berpotensi melakukan klaim. Nasabah yang berpotensi melakukan klaim adalah nasabah perempuan yang merokok, berapa banyak nasabah dengan ciri-ciri tersebut?

In [ ]:
# Cara 1
smoker_female_df = df.groupby('sex').smoker.value_counts()
smoker_female_df

sex     smoker
female  no        547
        yes       115
male    no        517
        yes       159
Name: smoker, dtype: int64

In [ ]:
# Cara 2
smoker_female_tab = pd.crosstab(df.sex, df.smoker)
smoker_female_tab

smoker,no,yes
sex,,
female,547,115
male,517,159


In [ ]:
# Cara 2
smoker_female_tab = pd.crosstab(df.sex, df.smoker)
smoker_female_tab

smoker,no,yes
sex,,
female,547,115
male,517,159


## 8. Rata-rata Biaya Klaim Nasabah No. 7
> Berdasarkan ciri-ciri nasabah di pertanyaan sebelumnya, berapa rata-rata biaya yang harus disiapkan perusahaan asuransi untuk membayarkan klaim nasabah-nasabah tersebut?

In [ ]:
# Menghitung rata-rata biaya klaim nasabah dengan kriteria tertentu
avg_claim_df = df[(df['sex'] == 'female') & (df['smoker'] == 'yes')]['charges'].mean()

print("Rata-rata biaya klaim nasabah dengan kriteria tertentu: $", avg_claim_df)
print("Rata-rata biaya klaim nasabah dengan kriteria tertentu: ${:.0f}".format(avg_claim_df))

Rata-rata biaya klaim nasabah dengan kriteria tertentu: $ 30678.996276260867
Rata-rata biaya klaim nasabah dengan kriteria tertentu: $30679


## 9. Create New Feature: Revenue
> Anggi dari perusahaan asuransi ingin mengunjungi nasabah yang berusia 64 tahun dan memiliki 1 orang anak. Bantulah Anggi untuk menemukan dimana wilayah tempat tinggal nasabah tersebut? (Pilih lebih dari 1)

In [ ]:
# Menampilkan nasabah yang berusia 64 tahun dan memiliki 1 orang anak
region_df = df[(df['age'] == 64) & (df['children'] == 1)]
region_df

,age,sex,bmi,children,smoker,region,charges
62,64,male,24.70,1,no,northwest,30166.61817
328,64,female,33.80,1,yes,southwest,47928.03000
418,64,male,39.16,1,no,southeast,14418.28040


## 10. Average of BMI in North
> Berapa rata-rata body mass index (bmi) untuk nasabah yang berada di wilayah yang mengandung kata "north"?

In [ ]:
# Menghitung rata-rata bmi
north_df = df[df['region'].str.contains('north')]
avg_north_df = north_df['bmi'].mean()

print('Rata-rata BMI nasabah yang berada di wilayah yang mengandung kata "north" adalah:', avg_north_df)
print('Rata-rata BMI nasabah yang berada di wilayah yang mengandung kata "north" adalah: {:.2f}'.format(avg_north_df))

Rata-rata BMI nasabah yang berada di wilayah yang mengandung kata "north" adalah: 29.186664098613253
Rata-rata BMI nasabah yang berada di wilayah yang mengandung kata "north" adalah: 29.19


## 11. Average of BMI Male Smoker
> Bagi nasabah laki-laki perokok dengan usia lebih dari 50 tahun, jika dibuat kolom baru dengan nilai bmi - 1.5, maka berapa rata-rata bmi baru tersebut?

In [ ]:
# Filter untuk nasabah laki-laki perokok dengan usia lebih dari 50 tahun
filter_male = df['sex'] == 'male'
filter_smoker = df['smoker'] == 'yes'
filter_age = df['age'] > 50
filtered_df = df[filter_male & filter_smoker & filter_age]

# Buat kolom baru dengan nilai BMI dikurangi 1.5
filtered_df['BMI_new'] = filtered_df['bmi'] - 1.5

# Hitung rata-rata dari kolom baru
bmi_new_mean = filtered_df['BMI_new'].mean()

print('Rata-rata BMI baru untuk nasabah laki-laki perokok dengan usia lebih dari 50 tahun adalah:', bmi_new_mean)
print('Rata-rata BMI baru untuk nasabah laki-laki perokok dengan usia lebih dari 50 tahun adalah: {:.2f}'.format(bmi_new_mean))

Rata-rata BMI baru untuk nasabah laki-laki perokok dengan usia lebih dari 50 tahun adalah: 30.690405405405407
Rata-rata BMI baru untuk nasabah laki-laki perokok dengan usia lebih dari 50 tahun adalah: 30.69


<ipython-input-13-25ba9da8e4b8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['BMI_new'] = filtered_df['bmi'] - 1.5
